# E5 Gathering all the context pieces into a vector memory store

## Get a 🔥 kernel ready with a vector memory store

We're going to create a kernel that includes an embedding generation service — so that we can best construct the context that will be used.

In [1]:
!python -m pip install semantic-kernel==0.3.4.dev0

In [4]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAITextEmbedding
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI services used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
    kernel.add_text_embedding_generation_service("ada", AzureTextEmbedding("text-embedding-ada-002", api_key, endpoint))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))
    kernel.add_text_embedding_generation_service("ada", OpenAITextEmbedding("text-embedding-ada-002", api_key, org_id))

kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

/Users/johnmaeda/Documents/GitHub/pizzashop-and-ai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10b94fac0>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x10b94fcd0>}

## Prepare a "semantic" 🧐 memory bank to be used by the AI

We store the SWOT information in "conventional" arrays in Python that are not "semantic" in nature. That said, they're still eminently useful for the kind of computations we'll play with.

In [5]:
strength_questions = ["What unique recipes or ingredients does the pizza shop use?","What are the skills and experience of the staff?","Does the pizza shop have a strong reputation in the local area?","Are there any unique features of the shop or its location that attract customers?", "Does the pizza shop have a strong reputation in the local area?", "Are there any unique features of the shop or its location that attract customers?"]
weakness_questions = ["What are the operational challenges of the pizza shop? (e.g., slow service, high staff turnover)","Are there financial constraints that limit growth or improvements?","Are there any gaps in the product offering?","Are there customer complaints or negative reviews that need to be addressed?"]
opportunities_questions = ["Is there potential for new products or services (e.g., catering, delivery)?","Are there under-served customer segments or market areas?","Can new technologies or systems enhance the business operations?","Are there partnerships or local events that can be leveraged for marketing?"]
threats_questions = ["Who are the major competitors and what are they offering?","Are there potential negative impacts due to changes in the local area (e.g., construction, closure of nearby businesses)?","Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)?","Is there any risk due to changes in regulations or legislation (e.g., health and safety, employment)?"]

strengths = [ "Unique garlic pizza recipe that wins top awards","Owner trained in Sicily at some of the best pizzerias","Strong local reputation","Prime location on university campus" ]
weaknesses = [ "High staff turnover","Floods in the area damaged the seating areas that are in need of repair","Absence of popular calzones from menu","Negative reviews from younger demographic for lack of hip ingredients" ]
opportunities = [ "Untapped catering potential","Growing local tech startup community","Unexplored online presence and order capabilities","Upcoming annual food fair" ]
threats = [ "Competition from cheaper pizza businesses nearby","There's nearby street construction that will impact foot traffic","Rising cost of cheese will increase the cost of pizzas","No immediate local regulatory changes but it's election season" ]

And to store the information semantically, we use the text embedding model added to the kernel to convert text into long vectors of numbers. We only need to do this once. But note that unless we have a "vector database" wired into the kernel, the embedding calculations do not last beyond this notebook. That's why it's called a "volatile" memory store.

In [6]:
memoryCollectionName = "SWOT"

for i in range(len(strengths)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"strength-{i}", text=f"Internal business strength (S in SWOT) that makes customers happy and satisfied Q&A: Q: {strength_questions[i]} A: {strengths[i]}")
for i in range(len(weaknesses)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"weakness-{i}", text=f"Internal business weakness (W in SWOT) that makes customers unhappy and dissatisfied Q&A: Q: {weakness_questions[i]} A: {weaknesses[i]}")
for i in range(len(opportunities)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"opportunity-{i}", text=f"External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: {opportunities_questions[i]} A: {opportunities[i]}")
for i in range(len(threats)):
    await kernel.memory.save_information_async(memoryCollectionName, id=f"threat-{i}", text=f"External threat (T in SWOT) to the business that impacts its survival Q&A: Q: {threats_questions[i]} A: {threats[i]}")

print("Embeddings for SWOT have been generated")

Embeddings for SWOT have been generated


Keep in mind that weaknesses and threats are v similar but strengths and weaknesses usually pertain to internal factors; whereas opportunities and threats pertain to external factors. Play around with the `ask` variable below to get a sense of how the embedding model does with comparing your `ask` to the vectors that have been computed and stored in the step above.

In [7]:
from IPython.display import display, Markdown

ask = "rising costs for the business"
counter = 0

memories = await kernel.memory.search_async(memoryCollectionName, ask, limit=5, min_relevance_score=0.77)

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    print(f"Similarity result {counter}:\n  >> ID: {memory.id}\n  Text: {memory.text}  Relevance: {memory.relevance}\n")

Similarity result 1:
  >> ID: threat-2
  Text: External threat (T in SWOT) to the business that impacts its survival Q&A: Q: Are there economic or industry trends that could impact the business negatively (e.g., increased ingredient costs)? A: Rising cost of cheese will increase the cost of pizzas  Relevance: 0.8115734986603766

Similarity result 2:
  >> ID: weakness-1
  Text: Internal business weakness (W in SWOT) that makes customers unhappy and dissatisfied Q&A: Q: Are there financial constraints that limit growth or improvements? A: Floods in the area damaged the seating areas that are in need of repair  Relevance: 0.793127063303838

Similarity result 3:
  >> ID: opportunity-2
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Can new technologies or systems enhance the business operations? A: Unexplored online presence and order capabilities  Relevance: 0.7845129507724604

Similarity result 4:
  >> ID: opportunity-0
  Text: External op

The idea of having a semantic memory bank like this available is when new information can get thrown into the bank, and relevant information can be easily pulled out on-demand.

## Storing all the Business Thinking and Design Thinking into semantic memory

Let's take everything we have gathered thus far and put it into the semantic memory bank

In [5]:
skillsDirectory = "./plugins-sk"

pluginBT = kernel.import_semantic_skill_from_directory(skillsDirectory, "BusinessThinking");

my_context = kernel.create_new_context()
my_context['business'] = 'makes pizzas'
my_context['strengths'] = ", ".join(strengths)
my_context['weaknesses'] = ", ".join(weaknesses)
my_context['opportunities'] = ", ".join(opportunities)
my_context['threats'] = ", ".join(threats)

print("Computing cost efficiency...")
costefficiency_result = await kernel.run_async(pluginBT["SeekCostEfficiency"], input_context=my_context)
costefficiency_str = str("## Suggestions for how to gain cost efficiencies\n" + str(costefficiency_result))
await kernel.memory.save_information_async(memoryCollectionName, id="costefficiency", text=costefficiency_str)

print("Computing time efficiency...")
timeefficiency_result = await kernel.run_async(pluginBT["SeekTimeEfficiency"], input_context=my_context)
timeefficiency_str = "## Suggestions for how to gain time efficiencies\n"+str(timeefficiency_result)
await kernel.memory.save_information_async(memoryCollectionName, id="timeefficiency", text=timeefficiency_str)

print("Computing biz 101 overview...")
biz101_result = await kernel.run_async(pluginBT["Biz101Overview"], input_context=my_context)
biz101_str = "# Biz 101 thinking based on SWOT analysis\n"+str(biz101_result)
await kernel.memory.save_information_async(memoryCollectionName, id="biz101", text=biz101_str)

customer_comments = """
Customer 1: The seats look really raggedy.
Customer 2: The garlic pizza is the best on this earth.
Customer 3: I've noticed that there's a new server every time I visit, and they're clueless.
Customer 4: Why aren't there calzones?
Customer 5: I love the garlic pizza and can't get it anywhere else.
Customer 6: The garlic pizza is exceptional.
Customer 7: Pizza can get a little messy so I prefer a calzone's neatness.
Customer 8: Why is the pizza so expensive?
Customer 9: There's no way to do online ordering.
Customer 10: Why is the seating so uncomfortable and dirty?
"""

print("Computing design thinking interim results from customer comments ...")
pluginDT = kernel.import_semantic_skill_from_directory(skillsDirectory, "DesignThinking");
my_result = await kernel.run_async(pluginDT["Empathize"], pluginDT["Define"], input_str=customer_comments)
designthinking_str = "# Categorized observations\n"+str(my_result)
await kernel.memory.save_information_async(memoryCollectionName, id="designthinking", text=designthinking_str)

print("Memory is flush with information :+)")

Computing cost efficiency...
Computing time efficiency...
Computing biz 101 overview...
Computing design thinking interim results from customer comments ...
Memory is flush with information :+)


## Put the accumulated memory to good use

In [38]:
ask = "How can the business make more money built upon its core strengths quickly?"
counter = 0

gathered_context = []

memories = await kernel.memory.search_async(memoryCollectionName, ask, limit=5, min_relevance_score=0.77)

for memory in memories:
    if counter == 0:
        related_memory = memory.text
    counter += 1
    gathered_context.append(memory.text + "\n")
    print(f"Similarity result {counter}:\n  >> ID: {memory.id}\n  Text: {memory.text}  Relevance: {memory.relevance}\n")



Similarity result 1:
  >> ID: opportunity-0
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Is there potential for new products or services (e.g., catering, delivery)? A: Untapped catering potential  Relevance: 0.7859603751036197

Similarity result 2:
  >> ID: opportunity-2
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Can new technologies or systems enhance the business operations? A: Unexplored online presence and order capabilities  Relevance: 0.7858114919880576

Similarity result 3:
  >> ID: opportunity-1
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Are there under-served customer segments or market areas? A: Growing local tech startup community  Relevance: 0.7801646402457444

Similarity result 4:
  >> ID: opportunity-3
  Text: External opportunity (O in SWOT) for the business to gain entirely new customers Q&A: Q: Are there partner

Now that we have some useful context scraped from our available memories, we can now ask a question and hope to get a useful answer back. If more context to give to the session is desired, just change the parameter `limit` in the previous code block to integrate more of the knowledge that has been accumulated.

In [28]:
ask = "How can the business make more money built upon its core strengths quickly?"
qna_function_inline = kernel.create_semantic_function("""
The following is a conversation with an AI assistant. The assistant uses simple vocabulary and sentence structures ool audience
AI: Sure, I can help. What's your question?
User: {{$input}}
AI: Let me respond as markdown bullets.""",
                                                    max_tokens=2000,
                                                    temperature=0.1,
                                                    top_p=0.5)


output = qna_function_inline(ask + "This is what I know so far and the response should be grounded in this information " + " ".join(gathered_context))

display(Markdown(f"### Helpful advice for you in response to ...\n>{ask}\n\n" + str(output)))

### Helpful advice for you in response to ...
>How can the business make more money built upon its core strengths quickly?

- To make more money built upon its core strengths quickly, the business can consider the following strategies:
- Explore online presence and order capabilities to attract new customers who prefer online shopping and delivery options.
- Tap into the catering potential by offering catering services to attract new customers and increase revenue.
- Leverage the prime location on the university campus to attract more customers, especially students and faculty members.
- Target the growing local tech startup community by understanding their needs and offering tailored products or services.
- Utilize the owner's skills and experience in Sicilian pizza-making to create a unique selling point and attract customers who appreciate authentic and high-quality pizza.

## A simpler, compact, and less powerful way to do the same thing

You can also create a PLUGIN in SK that can `recall` the single most-similar memory from the vector store. It doesn't include as much context as used in the previous example, but it shows how a complex idea can be expressed simply with the primitives within SK.

In [43]:
ask = "How can the business make more money built upon its core strengths quickly?"
compact_asker_inline = kernel.create_semantic_function("""
Q: "{{$user_input}} Be sure to only source the response from the following context information." 
Context: {{recall $user_input}}
A: My response as markdown bullets:""",
    max_tokens=2000,temperature=0.1,top_p=0.5)
my_context = kernel.create_new_context()
my_context['user_input'] = ask
my_context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = memoryCollectionName
my_context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8
output = compact_asker_inline(context=my_context)

display(Markdown(f"### Compact advice for you in response to ...\n>{ask}\n\n" + str(output)))

### Compact advice for you in response to ...
>How can the business make more money built upon its core strengths quickly?

Context: External opportunity (O in SWOT) for the business to gain entirely new customers

- Utilize new technologies and systems to enhance the business operations
- Develop and establish an online presence to reach a wider customer base
- Implement online order capabilities to provide convenience and attract more customers
- Invest in digital marketing strategies to promote the business and attract new customers
- Offer exclusive online promotions or discounts to incentivize customers to make purchases
- Collaborate with influencers or online platforms to increase brand visibility and attract new customers
- Conduct market research to identify the target audience and tailor marketing efforts accordingly
- Continuously monitor and analyze online customer feedback to improve products and services
- Provide exceptional customer service through online channels to build customer loyalty and attract new customers.

## Quick business review

Every business benefits from four courses of action to be used together or in combination:

1. Grow the existing business
2. Save money and time
3. Add completely new business
4. Prepare for the unknown

![](assets/shopkeeper.png)

The business owner is always leaking time and money from their valuable buckets of time and money. Let's help them succeed!

## Quick tech review

1. AI PLUGINs in SK enable encapsulating complex AI computations to be used on their own or to feed their results into other AI PLUGIN. 
2. AI memories, often called "vector databases," can be used to hold information that is semantically searchable and can be leveraged as data to get fed into an AI PLUGIN.
3. So-called "hallucinations" are reduced by building AI PLUGINs to process prompts, and also by managing banks of semantic information to assemble and deepen contexts to feed into AI computations.
4. AI PLUGINs and vector databases are used most effectively with AI PLANNERs. What's a PLANNER? We'll jump into that next.